In [ ]:
! pip install pyspark

In [20]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, regexp_extract

In [ ]:
# Conexão com o Google Drive realizada pela interface gráfica

In [21]:
spark = SparkSession.builder.getOrCreate()

In [22]:
produtos = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/produtos.csv', inferSchema=True, header=True)
clientes = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/clientes.csv', inferSchema=True, header=True)
itens_pedido = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/itens_pedido.csv', inferSchema=True, header=True)
pedidos = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pedidos.csv', inferSchema=True, header=True)

In [23]:
# Verificar se a tabela itens_pedido possui entradas com preço inválido
itens_pedido = itens_pedido.withColumn('preco_valido', when(col('preco') > 0, 'Válido').otherwise('Inválido'))

itens_pedido.select('id_pedido','id_produto', 'preco', 'preco_valido').show()
itens_pedido.filter(col('preco_valido') == 'Inválido').show()

+--------------------+--------------------+------+------------+
|           id_pedido|          id_produto| preco|preco_valido|
+--------------------+--------------------+------+------------+
|00010242fe8c5a6d1...|4244733e06e7ecb49...|  58.9|      Válido|
|00018f77f2f0320c5...|e5f2d52b802189ee6...| 239.9|      Válido|
|000229ec398224ef6...|c777355d18b72b67a...| 199.0|      Válido|
|00024acbcdf0a6daa...|7634da152a4610f15...| 12.99|      Válido|
|00042b26cf59d7ce6...|ac6c3623068f30de0...| 199.9|      Válido|
|00048cc3ae777c65d...|ef92defde845ab845...|  21.9|      Válido|
|00054e8431b9d7675...|8d4f2bb7e93e6710a...|  19.9|      Válido|
|000576fe39319847c...|557d850972a7d6f79...| 810.0|      Válido|
|0005a1a1728c9d785...|310ae3c140ff94b03...|145.95|      Válido|
|0005f50442cb953dc...|4535b0e1091c278df...| 53.99|      Válido|
|00061f2a7bc09da83...|d63c1011f49d98b97...| 59.99|      Válido|
|00063b381e2406b52...|f177554ea93259a5b...|  45.0|      Válido|
|0006ec9db01a64e59...|99a4788cb24856965.

In [24]:
# verificar se a tabela pedidos contém entradas com status inválido
status_permitidos = ['created', 'approved', 'delivered', 'shipped', 'canceled', 'invoiced', 'processing']

pedidos = pedidos.withColumn('status_valido', when(col('status_pedido').isin(status_permitidos), 'Válido').otherwise('Inválido'))

pedidos.select('id_pedido', 'status_pedido', 'status_valido').show()

pedidos.filter(col('status_valido') == 'Inválido').show()

+--------------------+-------------+-------------+
|           id_pedido|status_pedido|status_valido|
+--------------------+-------------+-------------+
|e481f51cbdc54678b...|    delivered|       Válido|
|53cdb2fc8bc7dce0b...|    delivered|       Válido|
|47770eb9100c2d0c4...|    delivered|       Válido|
|949d5b44dbf5de918...|    delivered|       Válido|
|ad21c59c0840e6cb8...|    delivered|       Válido|
|a4591c265e18cb1dc...|    delivered|       Válido|
|136cce7faa42fdb2c...|     invoiced|       Válido|
|6514b8ad8028c9f2c...|    delivered|       Válido|
|76c6e866289321a7c...|    delivered|       Válido|
|e69bfb5eb88e0ed6a...|    delivered|       Válido|
|e6ce16cb79ec1d90b...|    delivered|       Válido|
|34513ce0c4fab462a...|    delivered|       Válido|
|82566a660a982b15f...|    delivered|       Válido|
|5ff96c15d0b717ac6...|    delivered|       Válido|
|432aaf21d85167c2c...|    delivered|       Válido|
|dcb36b511fcac050b...|    delivered|       Válido|
|403b97836b0c04a62...|    deliv

In [25]:
for coluna in clientes.columns:
  clientes.select(count(col(coluna)).alias(coluna)).show()

# Verificar a quatidade de valores nulos na tabela clientes
for coluna in clientes.columns:
  clientes.select(count(when(col(coluna).isNull(), coluna)).alias(coluna)).show()

+----------+
|id_cliente|
+----------+
|     99441|
+----------+

+----------------+
|id_unico_cliente|
+----------------+
|           99441|
+----------------+

+-----------+
|cep_cliente|
+-----------+
|      99441|
+-----------+

+--------------+
|cidade_cliente|
+--------------+
|         99441|
+--------------+

+--------------+
|estado_cliente|
+--------------+
|         99441|
+--------------+

+----------+
|id_cliente|
+----------+
|         0|
+----------+

+----------------+
|id_unico_cliente|
+----------------+
|               0|
+----------------+

+-----------+
|cep_cliente|
+-----------+
|          0|
+-----------+

+--------------+
|cidade_cliente|
+--------------+
|             0|
+--------------+

+--------------+
|estado_cliente|
+--------------+
|             0|
+--------------+



In [26]:
# outra forma de verificar os nulos (com o for inline)
n_nulls_pedidos_df = pedidos.select([count(when(col(c).isNull(), c)).alias(c) for c in pedidos.columns])
n_nulls_pedidos_df.show()

+---------+----------+-------------+------------------+---------------------+-------------------------+--------------------+---------------------+-------------+
|id_pedido|id_cliente|status_pedido|data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|status_valido|
+---------+----------+-------------+------------------+---------------------+-------------------------+--------------------+---------------------+-------------+
|        0|         0|            0|                 0|                  160|                     1783|                2965|                    0|            0|
+---------+----------+-------------+------------------+---------------------+-------------------------+--------------------+---------------------+-------------+



In [27]:
# Verificar se existem pedidos com id de cliente inexistente com join
pedidos_integridade_df = pedidos.join(clientes, pedidos.id_cliente == clientes.id_cliente, 'left_anti')
pedidos_integridade_df.select('id_pedido', 'id_cliente').show()

+---------+----------+
|id_pedido|id_cliente|
+---------+----------+
+---------+----------+



In [28]:
itens_integridade_df = itens_pedido.join(produtos, itens_pedido.id_produto == produtos.id_produto, 'left_anti')
itens_integridade_df.select('id_pedido', 'id_produto').show()

+---------+----------+
|id_pedido|id_produto|
+---------+----------+
+---------+----------+



In [29]:
clientes_validacao_df = clientes.withColumn('cep_valido', when(col('cep_cliente').rlike(r'^d{5}$'), 'Válido').otherwise('Inválido'))  # https://regexr.com/
clientes_validacao_df.select('id_cliente', 'cep_cliente', 'cep_valido').show()

+--------------------+-----------+----------+
|          id_cliente|cep_cliente|cep_valido|
+--------------------+-----------+----------+
|06b8999e2fba1a1fb...|      14409|  Inválido|
|18955e83d337fd6b2...|       9790|  Inválido|
|4e7b3e00288586ebd...|       1151|  Inválido|
|b2b6027bc5c5109e5...|       8775|  Inválido|
|4f2d8ab171c80ec83...|      13056|  Inválido|
|879864dab9bc30475...|      89254|  Inválido|
|fd826e7cf63160e53...|       4534|  Inválido|
|5e274e7a0c3809e14...|      35182|  Inválido|
|5adf08e34b2e99398...|      81560|  Inválido|
|4b7139f34592b3a31...|      30575|  Inválido|
|9fb35e4ed6f0a14a4...|      39400|  Inválido|
|5aa9e4fdd4dfd2095...|      20231|  Inválido|
|b2d1536598b73a9ab...|      18682|  Inválido|
|eabebad39a88bb6f5...|       5704|  Inválido|
|1f1c7bf1c9b041b29...|      95110|  Inválido|
|206f3129c0e4d7d0b...|      13412|  Inválido|
|a7c125a0a07b75146...|      22750|  Inválido|
|c5c61596a3b6bd0ce...|       7124|  Inválido|
|9b8ce803689b3562d...|       5416|

In [32]:
produtos_validacao_df = produtos.withColumn('id_produto_valido', when(col('id_produto').rlike(r'^[a-f0-9]{32}$'), 'Válido').otherwise('inválido'))
produtos_validacao_df.select('id_produto', 'id_produto_valido').show()

+--------------------+-----------------+
|          id_produto|id_produto_valido|
+--------------------+-----------------+
|1e9e8ef04dbcff454...|           Válido|
|3aa071139cb16b67c...|           Válido|
|96bd76ec8810374ed...|           Válido|
|cef67bcfe19066a93...|           Válido|
|9dc1a7de274444849...|           Válido|
|41d3672d4792049fa...|           Válido|
|732bd381ad09e530f...|           Válido|
|2548af3e6e77a690c...|           Válido|
|37cc742be07708b53...|           Válido|
|8c92109888e8cdf9d...|           Válido|
|14aa47b7fe5c25522...|           Válido|
|03b63c5fc16691530...|           Válido|
|cf55509ea8edaaac1...|           Válido|
|7bb6f29c2be577161...|           Válido|
|eb31436580a610f20...|           Válido|
|3bb7f144022e67327...|           Válido|
|6a2fb4dd53d2cdb88...|           Válido|
|a1b71017a84f92fd8...|           Válido|
|a0736b92e52f6cead...|           Válido|
|f53103a77d9cf245e...|           Válido|
+--------------------+-----------------+
only showing top

In [33]:
if 'email_cliente' in clientes.columns:
  clientes_verifica_email_df = clientes.withColumn('email_valido', when(col('email_cliente').contains('@'), 'Válido').otherwise('Inválido'))
  clientes_verifica_email_df.select('id_cliente', 'email_cliente', 'email_valido').show()

In [34]:
spark.stop()